# Change Detection using classification


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 4 - Prac 1" notebook.

## Description
Many earth observation datasets are available at regular intervals over long periods of time. This enables us to detect changes on the Earth’s surface. In this session, you will learn how to apply change detection technique in remote sensing by conducting a supervised classification on two images composites seperated years apart.

## Aims of the practical session

* Apply classification algorithms to classify land cover classes before and after an event
* Comparing two classified images to see the changes during the time

## Load packages

Import GEE packages that are needed for the analysis.

In [ ]:
import ee
import geemap
# ee.Authenticate()

## Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [ ]:
Map = geemap.Map()
# Map.add_basemap('HYBRID')

## Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map. In this example, we'll import a geojson from github that marks the boundary of the ACT urban area.

In [ ]:
# # # load shapefile (ROI)
path = 'https://raw.githubusercontent.com/nicolasyounes/engn3903/week5update/figures/ACT_urban_region.geojson'
geometry = geemap.geojson_to_ee(path)
Map.addLayer(geometry, {}, 'Canberra region')
Map.centerObject(geometry);

## Load the training data
Training data (or a training dataset) is the initial data used to train machine learning models. Import your pre-selected training dataset from your system.

In [ ]:
#### load training data
training_path = "https://raw.githubusercontent.com/nicolasyounes/engn3903/main/figures/training_data.geojson"
training_data = geemap.geojson_to_ee(training_path)

#print how many classes there are in the TD
df = geemap.ee_to_geopandas(training_data)
n_classes = len(df['landcover'].unique())
print(f'There are {n_classes} landcover classes in the training dataset')

## Load and composite a Landsat 8 Image collection from 2014

In the next few code cells we will extract training data from Landsat 8 images over the region specified by the ROI

We will:
* Define a function for cloud masking and rescaling Landsat 8 images
* Load Landsat 8 images for the analysis
* Filter a collection by date range
* Calculate a temporal median to collapse the time dimension
* Clip based on the geometry

In [ ]:
def rescale_landsatC2(image):
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True)

def cloudMaskL8Collection2(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

  # Replace the original bands with the scaled ones and apply the masks.
    return image.updateMask(qaMask) \
          .updateMask(saturationMask)

In [ ]:
## collect before event images 
ls8_before = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(geometry)
    .filterDate('2014-01-01', '2014-12-31') ### Note: you can try different dates
    .map(rescale_landsatC2) #map the cloudmasking/rescaling function
    .map(cloudMaskL8Collection2)
    .median() #temporal composite
    .clip(geometry)
)

### Visualize the composite
* Clip based on the geometry
* Display it on Geemap

In [ ]:
vis_params = {'min': 0, 'max': 0.4, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}

#Map.centerObject(point, 8)
Map.addLayer(ls8_before, vis_params, "Landsat8_before")
Map.addLayerControl()
Map

## Extract pixel values from the image using the training data
Now that we have created the points and labels, we need to sample the imagery using image.sampleRegions(). This command will extract the reflectance in the designated bands for each of the points you have created. 

In [ ]:
# Overlay the point on the image to get training data.
training = ls8_before.sampleRegions(**{
  'collection': training_data,
  'properties': ['landcover'],
  'scale': 30
})

## Supervised Classification

The <a href="https://developers.google.com/earth-engine/guides/classification">Classifier</a> package in handles supervised classification by ML algorithms running in Earth Engine. Thus, in this part we will:
* Instantiate a supervised classifier
* Set its parameters if necessary
* Train the classifier using the training data
* Classify an image or feature collection
* Display the classified map

In [ ]:
# Train a classifier.
# # select bands wanted to use in the classification
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']

classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': bands
})

In [ ]:
# # Classify the image.
beforeClassified = ls8_before.classify(classifier)
Map.addLayer(beforeClassified,{'min': 0, 'max': 3, 'palette': ['yellow', 'green', 'red', 'blue']}, 'before_classified')
Map

## Load and composite an Image collection from 2021
We will:
* Load after Landsat-8 images for the analysis
* Filter a collection by date range
* Make a cloud-free composite 

In [ ]:
# # After Images.
ls8_after = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(geometry)
    .filterDate('2021-01-01', '2021-12-31')
    .map(rescale_landsatC2) #map the cloudmasking/rescaling function
    .map(cloudMaskL8Collection2)
    .median() #temporal composite
    .clip(geometry)
)

### Visualize the composite
* Clip based on the geometry
* Display it on Geemap

In [ ]:
Map.addLayer(ls8_after, vis_params, "Landsat8_after")
Map.addLayerControl()
Map

## Classify the later image collection
* Apply the same supervised classifier on after image
* Classify an image or feature collection
* Display the classified map

In [ ]:
# Classify the image.
afterClassified= ls8_after.classify(classifier)
Map.addLayer(afterClassified,{'min': 0, 'max': 3, 'palette': ['yellow', 'green', 'red', 'blue']}, 'after_classified')
Map

### Reclassify the images to make change detection easier

In the next step we will subtract one image from another to visualize areas that have changed. This is easier if we reclassify the 0-4 land cover classes to 1-4

In [ ]:
# Reclassify from 0-3 to 1-4
beforeClasses = beforeClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])
afterClasses = afterClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])

## Change Detection
We are dealing with multi-class images. A useful metric for change detection is to know how many pixels from class X changed to class Y. 
In the `changed` image below, areas mapped with a `1` have undergone a change in landcover class, areas with `0` are the same as earlier.

In [ ]:
# Show all changed areas
changed = afterClasses.subtract(beforeClasses).neq(0)
Map.addLayer(changed, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'Change')
Map

<div class="alert alert-block alert-danger"> 
    
## Exercise 1 

1. Using the `changed` image, calculate the total area that has undergone change (the sum of all values equal to 1 in the `changed` image, muliplied by the area of a pixel.) 
1. Create bar graphs showing the area of each class in both classified images. 
2. Discuss which classes changed and by how much. Were all the pixels that changed related to real changes in land use? Or were there errors in the change detection?
3. What other methods might we use to help us distinguish 'real' change from spurious change (or mere variability in a land cover class)?

<div>

## References
This is where the references go. For exmaple:

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
* "Earth Observation: Data, Processing and Applications" book. Available through Wattle, or http://www.crcsi.com.au/earth-observation-series.

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** August 2022